In [1]:
!pip install transformers==4.44.2 joblib==1.4.2 scikit-learn==1.6.0 numpy==1.26.4 pandas==2.2.3 scipy==1.13.1 seaborn==0.13.2 tqdm==4.66.5 lightgbm==4.5.0 xgboost==2.1.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 70.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 99.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 47.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 11.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 87.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Un

In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, Dataset


# Load datasets
train_df = pd.read_csv('/kaggle/input/mdck-set/Train_MDCK.csv')
train_df = train_df[['ID', 'SMILES', 'Permeability']]
test_df = pd.read_csv('/kaggle/input/mdck-set/Test_MDCK.csv')
test_df = test_df[['ID', 'SMILES', 'Permeability']]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
model = AutoModelForSequenceClassification.from_pretrained('seyonec/PubChem10M_SMILES_BPE_450k', num_labels=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/336M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/PubChem10M_SMILES_BPE_450k and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Custom dataset class
class SMILESDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=325):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        smiles = self.dataframe.iloc[idx]['SMILES']
        permeability = self.dataframe.iloc[idx]['Permeability']
        inputs = self.tokenizer(smiles, return_tensors='pt', padding="max_length", truncation=True, max_length=self.max_length)
        
        input_ids = inputs['input_ids'].squeeze(0)  # Shape: (sequence_length,)
        attention_mask = inputs['attention_mask'].squeeze(0)  # Shape: (sequence_length,)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(permeability, dtype=torch.float)
        }

In [5]:
# datasets
train_dataset = SMILESDataset(train_df, tokenizer)
test_dataset = SMILESDataset(test_df, tokenizer)
batch_size = 16
# data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 20

In [7]:
# Training loop
from tqdm import tqdm
for epoch in range(num_epochs):
    print(f"Entered Epoch {epoch + 1}")
    model.train()
    train_loss = 0

    for batch in tqdm(train_loader, desc=f'Training Epoch {epoch + 1}/{num_epochs}', unit='batch'):
        optimizer.zero_grad()

        # Move all batch tensors to device
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch["labels"].unsqueeze(1)  # still shape: (batch_size, 1)

        # Forward pass
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels
        )
        loss = outputs.loss
        train_loss += loss.item()

        # Backprop and optimizer step
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}')

Entered Epoch 1


Training Epoch 1/20: 100%|██████████| 4/4 [00:02<00:00,  1.79batch/s]


Epoch 1/20 - Train Loss: 17.1997
Entered Epoch 2


Training Epoch 2/20: 100%|██████████| 4/4 [00:01<00:00,  2.80batch/s]


Epoch 2/20 - Train Loss: 3.0801
Entered Epoch 3


...

Epoch 18/20 - Train Loss: 0.4601
Entered Epoch 19


Training Epoch 19/20: 100%|██████████| 4/4 [00:01<00:00,  2.62batch/s]


Epoch 19/20 - Train Loss: 0.3935
Entered Epoch 20


Training Epoch 20/20: 100%|██████████| 4/4 [00:01<00:00,  2.62batch/s]

Epoch 20/20 - Train Loss: 0.3190


In [8]:
# Saving the model after training
model_name = 'PubChem10M_SMILES_BPE_450k_model_1_mdck'
model_save_path = f'/kaggle/working/{model_name}'
os.makedirs(model_save_path, exist_ok=True)

tokenizer.save_pretrained(model_save_path)
model.save_pretrained(model_save_path)

print(f'Model and tokenizer saved to {model_save_path}')

Model and tokenizer saved to /kaggle/working/PubChem10M_SMILES_BPE_450k_model_1_mdck


In [9]:
from scipy.stats import pearsonr, spearmanr

model.eval()
test_loss = 0
test_true_labels = []
predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing', unit='batch'):
      
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].unsqueeze(1).to(device).float()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        test_loss += loss.item()

        test_true_labels.extend(labels.cpu().numpy())
        preds = outputs.logits.squeeze().cpu().numpy()  
        predictions.extend(preds)

# Final test loss
avg_test_loss = test_loss / len(test_loader)
print(f'Test Loss: {avg_test_loss:.4f}')

test_true_labels = np.array(test_true_labels).flatten()
predictions = np.array(predictions)
print(test_true_labels.shape)
print(predictions.shape)

# Performance metrics
mse = mean_squared_error(test_true_labels, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_true_labels, predictions)
r2 = r2_score(test_true_labels, predictions)
PCC,_ = pearsonr(test_true_labels, predictions)
SCC,_ = spearmanr(test_true_labels, predictions)

# Print performance metrics
print(f'Mean Squared Error: {mse:.4f}')
print(f'Root Mean Squared Error: {rmse:.4f}')
print(f'Mean Absolute Error: {mae:.4f}')
print(f'R^2 Score: {r2:.4f}')
print(f'Pearson Correlation Coefficient: {PCC:.4f}')
print(f'Spearman Correlation Coefficient: {SCC:.4f}')

# Print hyperparameters
print("Hyperparameters:")
print(f"Learning Rate: {5e-5}")
print(f"Batch Size: 16")
print(f"Epochs: {num_epochs}")

Testing: 100%|██████████| 1/1 [00:00<00:00,  7.41batch/s]

Test Loss: 0.4432
(13,)
(13,)
Mean Squared Error: 0.4432
Root Mean Squared Error: 0.6657
Mean Absolute Error: 0.5662
R^2 Score: 0.2466
Pearson Correlation Coefficient: 0.5199
Spearman Correlation Coefficient: 0.3659
Hyperparameters:
Learning Rate: 5e-05
Batch Size: 16
Epochs: 20


In [10]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model_name = 'PubChem10M_SMILES_BPE_450k_model_1_mdck'
model_save_path = f'/kaggle/working/{model_name}'

if not os.path.exists(model_save_path):
    raise FileNotFoundError(f"The model directory {model_save_path} does not exist.")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_save_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_save_path, trust_remote_code=True).to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at /kaggle/working/PubChem10M_SMILES_BPE_450k_model_1_mdck and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Load your datasets
train_df = pd.read_csv('/kaggle/input/mdck-set/Train_MDCK.csv')
train_df = train_df[['ID', 'SMILES', 'Permeability']]
test_df = pd.read_csv('/kaggle/input/mdck-set/Test_MDCK.csv')
test_df = test_df[['ID', 'SMILES', 'Permeability']]

In [12]:
train_encodings = tokenizer(list(train_df['SMILES']), truncation=True, padding=True, max_length=325, return_tensors="pt")
test_encodings = tokenizer(list(test_df['SMILES']), truncation=True, padding=True, max_length=325, return_tensors="pt")

In [13]:
from tqdm import tqdm 
batch_size = 16 

def generate_embeddings(encodings, batch_size):
    embeddings = []
    model.eval() 
    with torch.no_grad():
        for i in tqdm(range(0, len(encodings['input_ids']), batch_size), desc="Processing batches"):
            batch = {key: val[i:i + batch_size].to(device) for key, val in encodings.items()}  
            outputs = model(**batch)
            embeddings.append(outputs.last_hidden_state)
    return torch.cat(embeddings, dim=0)


In [14]:
train_embeddings = generate_embeddings(train_encodings, batch_size)
print(train_embeddings.shape)
train_embeddings = torch.mean(train_embeddings, dim=1)
print(train_embeddings.shape)

Processing batches: 100%|██████████| 4/4 [00:00<00:00, 16.08it/s]

torch.Size([51, 158, 768])
torch.Size([51, 768])


In [15]:
column_names = [f'x_fine_emb_pubchem{i}' for i in range(train_embeddings.shape[1])]
embeddings_df = pd.DataFrame(data=train_embeddings.cpu().numpy(), columns=column_names)
train_data = pd.concat([train_df, embeddings_df], axis=1)

In [16]:
test_embeddings = generate_embeddings(test_encodings, batch_size)
print(test_embeddings.shape)
test_embeddings = torch.mean(test_embeddings, dim=1)
print(test_embeddings.shape)

Processing batches: 100%|██████████| 1/1 [00:00<00:00, 109.08it/s]

torch.Size([13, 160, 768])
torch.Size([13, 768])


In [17]:
column_names = [f'x_fine_emb_pubchem{i}' for i in range(test_embeddings.shape[1])]
embeddings_df = pd.DataFrame(data=test_embeddings.cpu().numpy(), columns=column_names)
test_data = pd.concat([test_df, embeddings_df], axis=1)

In [18]:
train_data.to_csv("/kaggle/working/Train_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_mdck.csv",index=False)
test_data.to_csv("/kaggle/working/Test_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_mdck.csv",index=False)

In [19]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression  # LogisticRegression is not used for regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [20]:
train_data = pd.read_csv("/kaggle/working/Train_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_mdck.csv")
test_data = pd.read_csv("/kaggle/working/Test_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_mdck.csv")

In [21]:
def train_and_test_predict(models, X_train, y_train, X_test, y_test):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    results = {}
    predictions = []  

    for model in models:
        model_name = model.__class__.__name__
        predictions_train = []
        actual_y_train = []

        test_predictions_folds = []

        

        for train_index, val_index in kf.split(X_train):
            X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

            model.fit(X_train_fold, y_train_fold)

            y_pred_fold = model.predict(X_val_fold)
            y_pred_fold = np.clip(y_pred_fold, -10, -3.9)
            predictions_train.extend(y_pred_fold)
            actual_y_train.extend(y_val_fold)

            predictions_test_fold = model.predict(X_test)
            predictions_test_fold = np.clip(predictions_test_fold, -10, -3.9)
            test_predictions_folds.append(predictions_test_fold)


        mse_train = mean_squared_error(actual_y_train, predictions_train)
        mae_train = mean_absolute_error(actual_y_train, predictions_train)
        rmse_train = np.sqrt(mse_train)
        r2_train = r2_score(actual_y_train, predictions_train)
        pearson_train, _ = pearsonr(actual_y_train, predictions_train)
        spearman_train, _ = spearmanr(actual_y_train, predictions_train)


        predictions_test_mean = np.mean(test_predictions_folds, axis=0)
        predictions_test_std = np.std(test_predictions_folds, axis=0)

        mse_test = mean_squared_error(y_test, predictions_test_mean)
        mae_test = mean_absolute_error(y_test, predictions_test_mean)
        rmse_test = np.sqrt(mse_test)
        r2_test = r2_score(y_test, predictions_test_mean)
        print(r2_test)
        pearson_test, _ = pearsonr(y_test, predictions_test_mean)
        spearman_test, _ = spearmanr(y_test, predictions_test_mean)
        
        

        predictions.append({
            'Model': model_name,
            'Y Train pred': predictions_train,
            'Y Test actual': y_test,
            'Test prediction folds': test_predictions_folds,
            'Test Predictions Mean': predictions_test_mean,
            'Test Predictions Std': predictions_test_std,

        })

        results[model_name] = {
            'Train MSE (5 fold cv)': f"{mse_train:.4f}",
            'Train MAE (5 fold cv)': f"{mae_train:.4f}",
            'Train RMSE (5 fold cv)': f"{rmse_train:.4f}",
            'Train RMSE (5 fold cv)': f"{rmse_train:.4f}",
            'Train R2 (5 fold cv)': f"{r2_train:.4f}",
            'Train PCC (5 fold cv)': f"{pearson_train:.4f}",
            'Train SCC (5 fold cv)': f"{spearman_train:.4f}",
            'Test MSE': f"{mse_test:.4f}",
            'Test MAE': f"{mae_test:.4f}",
            'Test RMSE': f"{rmse_test:.4f}",
            'Test R2': f"{r2_test:.4f}",
            'Test Pearson Correlation': f"{pearson_test:.4f}",
            'Test Spearman Correlation': f"{spearman_test:.4f}",
        }

    results_df = pd.DataFrame(results).T
    predictions_df = pd.DataFrame(predictions)

    return results_df, predictions_df



In [22]:
X_train = train_data.drop(['ID','SMILES','Permeability'],axis=1)
y_train = train_data['Permeability']
print("X_train shape: ",X_train.shape)
print("y_train shape: ",y_train.shape)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

X_test = test_data.drop(['ID','SMILES','Permeability'],axis=1)
y_test = test_data['Permeability']
print("X_test shape: ",X_test.shape)
print("y_test shape: ",y_test.shape)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
models = [
    lgb.LGBMRegressor(objective='regression',metric='rmse',boosting_type='gbdt',num_leaves=31,learning_rate=0.05,random_state=42),
    DecisionTreeRegressor(random_state=42),
    RandomForestRegressor(n_jobs=-1, random_state=42),
    GradientBoostingRegressor(random_state=42),
    AdaBoostRegressor(random_state=42),
    xgb.XGBRegressor(random_state=42),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=42),
    LinearRegression(), 
    KNeighborsRegressor(n_neighbors=3),
    SVR(),  
    MLPRegressor(random_state=42)
]
result_df, prediction_df = train_and_test_predict(models, X_train,y_train, X_test,  y_test)
result_df

X_train shape:  (51, 768)
y_train shape:  (51,)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X_test shape:  (13, 768)
y_test shape:  (13,)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000935 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2535
[LightGBM] [Info] Number of data points in the train set: 40, number of used features: 169
[LightGBM] [Info] Start training from score -5.646081
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.4784,0.5645,0.6916,0.2605,0.5151,0.5155,0.4617,0.5599,0.6795,0.2150,0.4815,0.3714
DecisionTreeRegressor,0.8650,0.7336,0.9301,-0.3372,0.3658,0.3800,0.5506,0.6033,0.7420,0.0640,0.4514,0.3521
RandomForestRegressor,0.4886,0.5716,0.6990,0.2447,0.5094,0.5282,0.4499,0.5656,0.6708,0.2351,0.4907,0.3824
GradientBoostingRegressor,0.5317,0.6052,0.7292,0.1781,0.4867,0.4563,0.4737,0.5714,0.6883,0.1946,0.4791,0.3384
AdaBoostRegressor,0.5478,0.6023,0.7401,0.1532,0.4515,0.5053,0.4971,0.6056,0.7050,0.1549,0.4259,0.3604
XGBRegressor,0.6571,0.6505,0.8106,-0.0157,0.3964,0.3856,0.5276,0.6101,0.7264,0.1030,0.4017,0.3659
ExtraTreesRegressor,0.4702,0.5471,0.6857,0.2731,0.5336,0.5712,0.4761,0.5942,0.6900,0.1906,0.4698,0.3109
LinearRegression,2.9119,1.2046,1.7064,-3.5013,0.0451,0.0906,1.3898,1.0218,1.1789,-1.3627,0.1644,0.1541
KNeighborsRegressor,0.5537,0.6062,0.7441,0.1441,0.4516,0.4897,0.3855,0.5517,0.6209,0.3447,0.6018,0.4711
SVR,0.5429,0.5821,0.7368,0.1607,0.4388,0.4422,0.4772,0.5817,0.6908,0.1888,0.4602,0.4154


In [23]:
result_df

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.4784,0.5645,0.6916,0.2605,0.5151,0.5155,0.4617,0.5599,0.6795,0.2150,0.4815,0.3714
DecisionTreeRegressor,0.8650,0.7336,0.9301,-0.3372,0.3658,0.3800,0.5506,0.6033,0.7420,0.0640,0.4514,0.3521
RandomForestRegressor,0.4886,0.5716,0.6990,0.2447,0.5094,0.5282,0.4499,0.5656,0.6708,0.2351,0.4907,0.3824
GradientBoostingRegressor,0.5317,0.6052,0.7292,0.1781,0.4867,0.4563,0.4737,0.5714,0.6883,0.1946,0.4791,0.3384
AdaBoostRegressor,0.5478,0.6023,0.7401,0.1532,0.4515,0.5053,0.4971,0.6056,0.7050,0.1549,0.4259,0.3604
XGBRegressor,0.6571,0.6505,0.8106,-0.0157,0.3964,0.3856,0.5276,0.6101,0.7264,0.1030,0.4017,0.3659
ExtraTreesRegressor,0.4702,0.5471,0.6857,0.2731,0.5336,0.5712,0.4761,0.5942,0.6900,0.1906,0.4698,0.3109
LinearRegression,2.9119,1.2046,1.7064,-3.5013,0.0451,0.0906,1.3898,1.0218,1.1789,-1.3627,0.1644,0.1541
KNeighborsRegressor,0.5537,0.6062,0.7441,0.1441,0.4516,0.4897,0.3855,0.5517,0.6209,0.3447,0.6018,0.4711
SVR,0.5429,0.5821,0.7368,0.1607,0.4388,0.4422,0.4772,0.5817,0.6908,0.1888,0.4602,0.4154


In [24]:
prediction_df

,Model,Y Train pred,Y Test actual,Test prediction folds,Test Predictions Mean,Test Predictions Std
0,LGBMRegressor,"[-5.27004134198572, -6.26664684177134, -5.4829...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.299852166075525, -5.27004134198572, -5.85...","[-5.261275875531089, -5.370224438292487, -5.57...","[0.128781337927666, 0.10555707647025636, 0.156..."
1,DecisionTreeRegressor,"[-5.82, -6.929917083, -5.167474653, -5.1, -5.1...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.82, -4.61, -6.929917083, -5.509833662, -6...","[-5.378, -4.878, -6.1744231302, -5.9394139734,...","[0.6047941798661758, 0.47368343859586226, 0.44..."
2,RandomForestRegressor,"[-5.181963068730004, -6.294134961160003, -5.44...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.068062035900004, -5.075144036520005, -5.8...","[-5.281566271376002, -5.423459609866002, -5.66...","[0.15083805839914724, 0.22984556988041713, 0.1..."
3,GradientBoostingRegressor,"[-5.042478743825043, -6.620542206718425, -5.23...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-4.926574328788401, -4.908707828379627, -6.3...","[-5.335474641001875, -5.493765712469684, -5.79...","[0.40687517434864057, 0.3646187832349236, 0.32..."
4,AdaBoostRegressor,"[-5.1042495572222215, -6.087718280199999, -5.3...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-4.994304357, -5.0793558854444445, -5.735, -...","[-5.2060369656444445, -5.549211359427778, -5.6...","[0.13267621582665443, 0.3459561319287336, 0.19..."
5,XGBRegressor,"[-4.7654333, -6.677271, -5.1480694, -5.3692245...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-4.935313, -4.8447113, -6.047671, -5.2744117...","[-5.2333803, -5.3639436, -5.921853, -5.6694527...","[0.32697693, 0.35756886, 0.22456552, 0.3119181..."
6,ExtraTreesRegressor,"[-5.084670258270002, -6.363089840660004, -5.31...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-4.929517790100005, -5.1147055600900035, -5....","[-5.188435495762003, -5.580615455192001, -5.66...","[0.2344693164969196, 0.24979701375490887, 0.13..."
7,LinearRegression,"[-4.447334414787026, -3.9, -3.9, -3.9, -4.7630...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-10.0, -5.889189227891415, -3.9, -3.9, -8.09...","[-7.6194648786507, -5.603654192454608, -5.9162...","[1.9640179079066231, 0.5019239649264065, 1.053..."
8,KNeighborsRegressor,"[-5.123333333333334, -5.756470717666667, -5.14...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-4.923333333333333, -5.359999999999999, -5.2...","[-5.334666666666665, -5.549333333333332, -5.35...","[0.2679021380197544, 0.19980434874591144, 0.16..."
9,SVR,"[-5.043390261582838, -6.362421034097634, -5.14...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-4.968109463610033, -5.030569000902871, -5.5...","[-5.3076451171804555, -5.377040222514305, -5.5...","[0.27984283145373395, 0.2781215597400838, 0.05..."


In [25]:
result_df.to_csv('/kaggle/working/Results_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_mdck.csv')
prediction_df.to_csv('/kaggle/working/Prediction_data_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_mdck.csv')

In [26]:
result_df

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.4784,0.5645,0.6916,0.2605,0.5151,0.5155,0.4617,0.5599,0.6795,0.2150,0.4815,0.3714
DecisionTreeRegressor,0.8650,0.7336,0.9301,-0.3372,0.3658,0.3800,0.5506,0.6033,0.7420,0.0640,0.4514,0.3521
RandomForestRegressor,0.4886,0.5716,0.6990,0.2447,0.5094,0.5282,0.4499,0.5656,0.6708,0.2351,0.4907,0.3824
GradientBoostingRegressor,0.5317,0.6052,0.7292,0.1781,0.4867,0.4563,0.4737,0.5714,0.6883,0.1946,0.4791,0.3384
AdaBoostRegressor,0.5478,0.6023,0.7401,0.1532,0.4515,0.5053,0.4971,0.6056,0.7050,0.1549,0.4259,0.3604
XGBRegressor,0.6571,0.6505,0.8106,-0.0157,0.3964,0.3856,0.5276,0.6101,0.7264,0.1030,0.4017,0.3659
ExtraTreesRegressor,0.4702,0.5471,0.6857,0.2731,0.5336,0.5712,0.4761,0.5942,0.6900,0.1906,0.4698,0.3109
LinearRegression,2.9119,1.2046,1.7064,-3.5013,0.0451,0.0906,1.3898,1.0218,1.1789,-1.3627,0.1644,0.1541
KNeighborsRegressor,0.5537,0.6062,0.7441,0.1441,0.4516,0.4897,0.3855,0.5517,0.6209,0.3447,0.6018,0.4711
SVR,0.5429,0.5821,0.7368,0.1607,0.4388,0.4422,0.4772,0.5817,0.6908,0.1888,0.4602,0.4154


In [27]:
prediction_df

,Model,Y Train pred,Y Test actual,Test prediction folds,Test Predictions Mean,Test Predictions Std
0,LGBMRegressor,"[-5.27004134198572, -6.26664684177134, -5.4829...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.299852166075525, -5.27004134198572, -5.85...","[-5.261275875531089, -5.370224438292487, -5.57...","[0.128781337927666, 0.10555707647025636, 0.156..."
1,DecisionTreeRegressor,"[-5.82, -6.929917083, -5.167474653, -5.1, -5.1...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.82, -4.61, -6.929917083, -5.509833662, -6...","[-5.378, -4.878, -6.1744231302, -5.9394139734,...","[0.6047941798661758, 0.47368343859586226, 0.44..."
2,RandomForestRegressor,"[-5.181963068730004, -6.294134961160003, -5.44...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-5.068062035900004, -5.075144036520005, -5.8...","[-5.281566271376002, -5.423459609866002, -5.66...","[0.15083805839914724, 0.22984556988041713, 0.1..."
3,GradientBoostingRegressor,"[-5.042478743825043, -6.620542206718425, -5.23...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-4.926574328788401, -4.908707828379627, -6.3...","[-5.335474641001875, -5.493765712469684, -5.79...","[0.40687517434864057, 0.3646187832349236, 0.32..."
4,AdaBoostRegressor,"[-5.1042495572222215, -6.087718280199999, -5.3...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-4.994304357, -5.0793558854444445, -5.735, -...","[-5.2060369656444445, -5.549211359427778, -5.6...","[0.13267621582665443, 0.3459561319287336, 0.19..."
5,XGBRegressor,"[-4.7654333, -6.677271, -5.1480694, -5.3692245...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-4.935313, -4.8447113, -6.047671, -5.2744117...","[-5.2333803, -5.3639436, -5.921853, -5.6694527...","[0.32697693, 0.35756886, 0.22456552, 0.3119181..."
6,ExtraTreesRegressor,"[-5.084670258270002, -6.363089840660004, -5.31...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-4.929517790100005, -5.1147055600900035, -5....","[-5.188435495762003, -5.580615455192001, -5.66...","[0.2344693164969196, 0.24979701375490887, 0.13..."
7,LinearRegression,"[-4.447334414787026, -3.9, -3.9, -3.9, -4.7630...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-10.0, -5.889189227891415, -3.9, -3.9, -8.09...","[-7.6194648786507, -5.603654192454608, -5.9162...","[1.9640179079066231, 0.5019239649264065, 1.053..."
8,KNeighborsRegressor,"[-5.123333333333334, -5.756470717666667, -5.14...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-4.923333333333333, -5.359999999999999, -5.2...","[-5.334666666666665, -5.549333333333332, -5.35...","[0.2679021380197544, 0.19980434874591144, 0.16..."
9,SVR,"[-5.043390261582838, -6.362421034097634, -5.14...",0 -6.300000 1 -5.350000 2 -6.200000 3...,"[[-4.968109463610033, -5.030569000902871, -5.5...","[-5.3076451171804555, -5.377040222514305, -5.5...","[0.27984283145373395, 0.2781215597400838, 0.05..."
